### Importar librerias

In [69]:
import pandas as pd
import requests
import time 
import random
pd.options.mode.chained_assignment = None

In [70]:

def format_float(value):
    return f'{value:,.2f}'

pd.options.display.float_format = format_float

url_saldos_totales = "https://riskzone.anywhereportfolio.com.ar:9099/api/saldosconsolidados"


In [71]:
tokenobj = {'key': 'value'}
REQUEST_GENERAL = requests.get(url_saldos_totales, json = tokenobj)
data  = (REQUEST_GENERAL.json())
df = pd.DataFrame(data)

### Dejar solo finalidad márgenes

In [72]:
df = df[df["FinalidadID"] == 2]


In [73]:
df[df["MiembroCompensadorCodigo"] == "274"]

,MiembroCompensadorID,CuentaCompensacionID,FinalidadID,MonedaID,MonedaDescripcion,Cotizacion,MiembroCompensadorCodigo,CuentaCompensacionCodigo,FinalidadDescripcion,EgresoNoVerificado,...,SaldoInicialMoneda,EgresoVerificadoMoneda,IngresoVerificadoMoneda,CuitCuil,CuentaNeteoDescripcion,MargenRequeridoAnterior,MargenRequeridoDelDia,MargenRequeridoTotal,MargenRequeridoAnteriorSC,HasCuentaInfoEmpty
2134,274,6702,2,2,Pesos,1.00,274,6702,Márgenes,0.00,...,"9,997,486,202.56",0.00,0.00,30500003193,BANCO BBVA ARGENTINA S.A.,"-9,908,998,500.00","1,522,881,000.00","-8,386,117,500.00","-9,908,998,500.00",False


### Definir saldos finales

In [74]:
df["SaldoInicialAP5"] = df["SaldoInicial"] + df["MargenRequeridoAnterior"]
df["SaldoFinalVerificado"] = df["SaldoInicialAP5"] + df["IngresoVerificado"] + df["EgresoVerificado"] #+ df["MargenRequeridoTotal"]
df["SaldoFinalNoVerificado"] = df["SaldoFinalVerificado"] + df["IngresoNoVerificado"] + df["EgresoNoVerificado"]


In [75]:
df[df["CuentaNeteoCodigo"] == "78391"]
df = df[["MiembroCompensadorCodigo","MiembroCompensadorID","CuentaCompensacionCodigo","CuentaCompensacionID","CuentaNeteoCodigo","MonedaDescripcion","SaldoInicialAP5","SaldoFinalVerificado","SaldoFinalNoVerificado","MargenRequeridoDelDia"]]
agg_functions = {'MiembroCompensadorCodigo': 'first', 'CuentaCompensacionCodigo': 'first', 'SaldoInicialAP5': 'sum', "SaldoFinalVerificado":"sum","SaldoFinalNoVerificado":"sum", "MargenRequeridoDelDia":"sum"}
df_pesos = df.groupby(df['CuentaNeteoCodigo']).aggregate(agg_functions).reset_index()


In [76]:
df_pesos[df_pesos["MiembroCompensadorCodigo"] == "274"]

,CuentaNeteoCodigo,MiembroCompensadorCodigo,CuentaCompensacionCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia
3513,6702,274,6702,"88,487,702.56","88,487,702.56","88,487,702.56","1,522,881,000.00"


In [77]:
saldo_negativo = df_pesos #[df_pesos["SaldoFinalNoVerificado"] < 0]

saldo_negativo["LOP_Verificado"] = saldo_negativo["SaldoFinalVerificado"] + saldo_negativo["MargenRequeridoDelDia"]
saldo_negativo["LOP_NoVerificado"] = saldo_negativo["SaldoFinalNoVerificado"] + saldo_negativo["MargenRequeridoDelDia"]

saldo_negativo[saldo_negativo["MiembroCompensadorCodigo"] == "274"]

,CuentaNeteoCodigo,MiembroCompensadorCodigo,CuentaCompensacionCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado
3513,6702,274,6702,"88,487,702.56","88,487,702.56","88,487,702.56","1,522,881,000.00","1,611,368,702.56","1,611,368,702.56"


In [78]:
url = "https://riskzone.anywhereportfolio.com.ar:9099/api/cuentas/getcuentacompensacion"
df = requests.get(url, verify=False)
data_cuentas  = (df.json())
data_cuentas = pd.DataFrame(data_cuentas)

def get_tipo_de_cuenta(CIMCODIGO):
    CIMCODIGO = str(CIMCODIGO)
    data_mc = ((data_cuentas[data_cuentas["CuentaCompensacionCodigo"] == CIMCODIGO]))
    tipocuenta = data_mc["TipoCuentaCompensacionDescripcion"].iloc[0] #.to_string(Name=False,dtype=False,index=False)
    return(tipocuenta)

def get_mc_name(MC):
    MC = str(MC)
    data_mc = ((data_cuentas[data_cuentas["MiembroCompensadorCodigo"] == MC]))
    name_mc = data_mc["MiembroCompensadorDescripcion"].iloc[0] #.to_string(Name=False,dtype=False,index=False)
    return(name_mc)

C:\Users\aggonzalez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'riskzone.anywhereportfolio.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [79]:
#saldo_negativo.columns
#saldo_negativo["LOP_Verificado"] = saldo_negativo["SaldoFinalVerificado"] + saldo_negativo["MargenRequeridoDelDia"]
#saldo_negativo["LOP_NoVerificado"] = saldo_negativo["SaldoFinalNoVerificado"] + saldo_negativo["MargenRequeridoDelDia"]

In [80]:
saldo_negativo

,CuentaNeteoCodigo,MiembroCompensadorCodigo,CuentaCompensacionCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado
0,100053,405,1405,"25,174.82","25,174.82","25,174.82",0.00,"25,174.82","25,174.82"
1,100062,205,51000,"7,900,000.00","7,900,000.00","7,900,000.00",0.00,"7,900,000.00","7,900,000.00"
2,1001,320,1001,"1,586,105.50","1,586,105.50","1,586,105.50",0.00,"1,586,105.50","1,586,105.50"
3,100107,269,1269,"-51,038.53","8,654.65","8,654.65","90,000.00","98,654.65","98,654.65"
4,100115,345,1345,"85,432.96","85,350.03","85,350.03",0.00,"85,350.03","85,350.03"
...,...,...,...,...,...,...,...,...,...
4979,99968,378,1378,"1,439,213.67","1,439,213.67","1,439,213.67","-25,928.27","1,413,285.40","1,413,285.40"
4980,99976,269,1269,"16,395.26","16,395.26","16,395.26","27,061.57","43,456.83","43,456.83"
4981,99978,345,1345,"50,825.25","50,825.25","50,825.25",0.00,"50,825.25","50,825.25"
4982,99985,133,1133,"42,962.50","42,962.50","42,962.50",0.00,"42,962.50","42,962.50"


In [81]:
tiposdecuenta = []
for i in range(len(saldo_negativo)):
    CIMCODIGO = (saldo_negativo.iloc[i,2])
    tipo = (get_tipo_de_cuenta(str(CIMCODIGO)))
    tiposdecuenta.append(tipo)
saldo_negativo["TipoDeCuenta"] = tiposdecuenta

nombreMC = []

for i in range(len(saldo_negativo)):
        MC_COD = (saldo_negativo.iloc[i,1])
        nombre_mc = get_mc_name(MC_COD)
        nombreMC.append(nombre_mc)

saldo_negativo["ALyC"] = nombreMC

first_column = saldo_negativo.pop('ALyC')
  
# insert column using insert(position,column_name,first_column) function
saldo_negativo.insert(0, 'ALyC', first_column)


### Obtener cual es la cuenta propia del MC

In [82]:
url = "https://riskzone.anywhereportfolio.com.ar:9099/api/cuentas/getcuentacompensacion"
df = requests.get(url, verify=False)
data_cuentas  = (df.json())
data_cuentas = pd.DataFrame(data_cuentas)


def get_cim_propia(MC):
    MC = str(MC)
    data_mc = ((data_cuentas[data_cuentas["MiembroCompensadorCodigo"] == (MC)]))
    propia = data_mc[data_mc["EsCuentaPropia"] == True]
    cim_codigo_propia = (propia["CuentaCompensacionCodigo"].to_string(name=False,dtype=False,index=False))
    return(cim_codigo_propia)


C:\Users\aggonzalez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'riskzone.anywhereportfolio.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [83]:


cuentapropia = []

for i in range(len(saldo_negativo)):
        MC = (saldo_negativo.iloc[i,2])
        cimpropia = get_cim_propia((MC))
        cuentapropia.append(cimpropia)

saldo_negativo["CuentaPropiaMc"] = cuentapropia


In [84]:
def cim_to_id(cim):

    cim = str(cim)

    cimcod = data_cuentas[data_cuentas["CuentaCompensacionCodigo"] == cim]
    cim = cimcod["CuentaCompensacionID"].to_string(index=False,dtype=False)
    return(cim)

def mccod_to_id(mc):

    mc = str(mc)

    mccod = data_cuentas[data_cuentas["MiembroCompensadorCodigo"] == mc]
    mc = mccod["MiembroCompensadorID"].to_string(index=False,dtype=False)
    mc = mc.split("\n")
    mc = mc[0]
    return(mc)


In [85]:
#saldo_negativo = saldo_negativo.rename(columns={"SaldoFinalVerificado" : "LOP_Verificado","SaldoFinalNoVerificado":"LOP_NoVerificado"})




In [86]:
dfsaldos = saldo_negativo[["MiembroCompensadorCodigo","CuentaPropiaMc"]]
dfsaldos = dfsaldos.drop_duplicates()



In [87]:

listasaldospropios = []
for i in range(len(dfsaldos)):
    try:
        ALYCCOD = (dfsaldos.iloc[i,0])
        ALYCID = mccod_to_id(ALYCCOD)
        PROPA = (dfsaldos.iloc[i,1])        
        FINALIDAD = 2
        PROPA = cim_to_id(PROPA)

        e = {'SaldoInicial':'sum', 'MargenRequeridoTotal':'sum'}

        tokenobj = {'key': 'value'}
        URLSALDO = f"https://riskzone.anywhereportfolio.com.ar:9099/api/saldosconsolidados?MCId={ALYCID}&CelID={PROPA}"
        q = requests.get(URLSALDO, json = tokenobj)
        SALDOALYC  = (q.json())
        SALDOALYC = pd.DataFrame(SALDOALYC)

        saldoprevia = (SALDOALYC[['MiembroCompensadorID', 'CuentaCompensacionID', "MonedaDescripcion",'FinalidadID',"SaldoInicial","MargenRequeridoTotal"]]) #cambiar Anterior por Total
        saldoprevia["AP5"] = saldoprevia["SaldoInicial"] + saldoprevia["MargenRequeridoTotal"] #CAMBIAR ANTERIOR POR Total
        saldoprevia = saldoprevia.groupby("FinalidadID").agg(e)
        saldoprevia = saldoprevia.reset_index()

        saldoprevia = (saldoprevia[saldoprevia["FinalidadID"] == FINALIDAD])
        saldoprevia["AP5"] = saldoprevia["SaldoInicial"] + saldoprevia["MargenRequeridoTotal"] #CAMBIAR ANTERIOR POR Total

        calditos = saldoprevia["AP5"].sum()
        saldodelapropia = calditos
    except:
        saldodelapropia = ALYCCOD
    
    listasaldospropios.append(saldodelapropia)

In [88]:
dfsaldos["SaldoPropia"] = listasaldospropios

dfsaldos = dfsaldos[["CuentaPropiaMc","SaldoPropia"]]
saldo_negativo = saldo_negativo.merge(dfsaldos, on = "CuentaPropiaMc", how = "left")

In [89]:
primer_filtro = saldo_negativo[saldo_negativo["TipoDeCuenta"] == "Terceros"]
e = {'LOP_NoVerificado':'sum', 'LOP_Verificado':'sum',"SaldoPropia":"first"}
primerfiltro = primer_filtro.groupby("MiembroCompensadorCodigo").agg(e)
primerfiltro["Resultado"] = primerfiltro["LOP_Verificado"] + primerfiltro["SaldoPropia"].astype(float)
cubiertosporalyc = primerfiltro[primerfiltro["Resultado"] > 0]
listacubiertos = list(cubiertosporalyc.index)
print(listacubiertos)

['117', '125', '131', '133', '136', '146', '155', '156', '159', '16', '160', '162', '163', '164', '165', '168', '170', '171', '172', '177', '181', '185', '187', '204', '205', '21', '218', '22', '23', '258', '26', '269', '284', '30', '303', '312', '313', '320', '321', '322', '323', '324', '325', '326', '327', '328', '329', '332', '334', '335', '336', '337', '338', '339', '340', '341', '343', '344', '345', '346', '349', '35', '350', '352', '354', '356', '357', '360', '361', '362', '363', '364', '365', '366', '367', '372', '373', '375', '376', '378', '379', '380', '381', '382', '383', '388', '392', '394', '395', '397', '398', '399', '401', '402', '405', '406', '409', '410', '411', '413', '415', '417', '418', '419', '42', '420', '424', '425', '426', '427', '429', '437', '438', '440', '445', '446', '451', '46', '461', '466', '469', '47', '475', '476', '478', '51', '526', '527', '53', '530', '535', '536', '54', '56', '62', '64', '66', '74', '78', '79', '802', '810', '84', '85', '86', '92']


In [90]:
cubiertosporalyc = saldo_negativo[saldo_negativo["MiembroCompensadorCodigo"].isin(listacubiertos) == True]


### AHORA SACAR DEL DATAFRAME ENTERO LOS QUE SI SON CUBIERTOS POR EL ALYC

In [91]:
NOCUBIERTOS = saldo_negativo[~saldo_negativo.isin(cubiertosporalyc)].dropna()


### test de agrupamiento

In [92]:
TESTI = NOCUBIERTOS.groupby(["ALyC","CuentaCompensacionCodigo","CuentaNeteoCodigo"]).sum()
#TESTI.to_excel("TestAgrupamiento.xls")
TESTI

SaldoInicialAP5  \
ALyC                                    CuentaCompensacionCodigo CuentaNeteoCodigo                     
ARCOR S A I C                           9605                     9605                  41,369,540.53   
ARPENTA VALORES S.A                     21382                    21382                    530,395.52   
BANCO BBVA ARGENTINA S.A.               6702                     6702                  88,487,702.56   
BANCO CENTRAL DE LA REPUBLICA ARGENTINA 6515                     6515              20,485,824,400.00   
BANCO CMF SOCIEDAD ANONIMA              33976                    33976                110,672,940.00   
...                                                                                              ...   
PROSECURITIES S.A                       1448                     168243                         0.00   
                                                                 17291                 20,405,747.48   
                                                                 42449                119,412,125.00   
VITERRA ARGENTINA S.A.                  142875                   142875                12,166,042.27   
YPF SOCIEDAD ANONIMA                    13806                    13806                918,500,464.70   

                                                                                    SaldoFinalVerificado  \
ALyC                                    CuentaCompensacionCodigo CuentaNeteoCodigo                         
ARCOR S A I C                           9605                     9605                      43,534,600.72   
ARPENTA VALORES S.A                     21382                    21382                        530,395.52   
BANCO BBVA ARGENTINA S.A.               6702                     6702                      88,487,702.56   
BANCO CENTRAL DE LA REPUBLICA ARGENTINA 6515                     6515                  20,485,824,400.00   
BANCO CMF SOCIEDAD ANONIMA              33976                    33976                    110,672,940.00   
...                                                                                                  ...   
PROSECURITIES S.A                       1448                     168243                             0.00   
                                                                 17291                              0.00   
                                                                 42449                            120.40   
VITERRA ARGENTINA S.A.                  142875                   142875                    12,166,042.27   
YPF SOCIEDAD ANONIMA                    13806                    13806                    911,936,296.44   

                                                                                    SaldoFinalNoVerificado  \
ALyC                                    CuentaCompensacionCodigo CuentaNeteoCodigo                           
ARCOR S A I C                           9605                     9605                        43,534,600.72   
ARPENTA VALORES S.A                     21382                    21382                          530,395.52   
BANCO BBVA ARGENTINA S.A.               6702                     6702                        88,487,702.56   
BANCO CENTRAL DE LA REPUBLICA ARGENTINA 6515                     6515                    20,485,824,400.00   
BANCO CMF SOCIEDAD ANONIMA              33976                    33976                      110,672,940.00   
...                                                                                                    ...   
PROSECURITIES S.A                       1448                     168243                               0.00   
                                                                 17291                                0.00   
                                                                 42449                              120.40   
VITERRA ARGENTINA S.A.                  142875                   142875                      12,166,042.27   
YPF SOC

In [93]:
NOCUBIERTOS[NOCUBIERTOS["MiembroCompensadorCodigo"] == 274]

,ALyC,CuentaNeteoCodigo,MiembroCompensadorCodigo,CuentaCompensacionCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,TipoDeCuenta,CuentaPropiaMc,SaldoPropia


### ¿Da saldo, recibe saldo?

In [94]:

url = "https://rm4api.testing.acsa:8099/api/cuentas/getcuentacompensacion"
#url = "https://riskzone.anywhereportfolio.com.ar:9099/api/cuentas/getcuentacompensacion"
test = requests.get(url, verify=False)
dasaldo  = (test.json())
dasaldo = pd.DataFrame(dasaldo)
dasaldo = dasaldo[["CuentaCompensacionCodigo","EntidadBursatilDescripcion","DaCreditoIntraCuentaCompensacion","RecibeCreditoIntraCuentaCompensacion","DaCreditoInterCuentaCompensacion","RecibeCreditoInterCuentaCompensacion"]]
dasaldo

C:\Users\aggonzalez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rm4api.testing.acsa'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,CuentaCompensacionCodigo,EntidadBursatilDescripcion,DaCreditoIntraCuentaCompensacion,RecibeCreditoIntraCuentaCompensacion,DaCreditoInterCuentaCompensacion,RecibeCreditoInterCuentaCompensacion
0,2016,MtR,True,True,True,False
1,1016,MtR,False,False,False,True
2,2021,MtR,True,True,True,False
3,1021,MtR,False,False,False,True
4,1022,MtR,False,False,False,True
...,...,...,...,...,...,...
574,1187,MtR,False,False,False,True
575,1499,UFEX,False,False,False,False
576,165590,UFEX,False,False,False,False
577,165600,UFEX,False,False,False,False


### MERGEAR EL DF CON DA_SALDO

In [95]:
LOPS = (NOCUBIERTOS.merge(dasaldo, how='left', on= "CuentaCompensacionCodigo"))

### FILTRAR LAS QUE TENGAN SALDOS NEGATIVOS MAYORES A 100 PESOS

In [96]:
LOPS = LOPS[LOPS["LOP_Verificado"] < -100]
LOPS = LOPS.reset_index()
LOPS = LOPS.drop("index", axis=1)


In [97]:
(LOPS)

,ALyC,CuentaNeteoCodigo,MiembroCompensadorCodigo,CuentaCompensacionCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,TipoDeCuenta,CuentaPropiaMc,SaldoPropia,EntidadBursatilDescripcion,DaCreditoIntraCuentaCompensacion,RecibeCreditoIntraCuentaCompensacion,DaCreditoInterCuentaCompensacion,RecibeCreditoInterCuentaCompensacion
0,NEXO AGENTE DE LIQUIDACION Y COMPENSACION INTE...,143173,393,35417,0.00,0.00,0.00,"-292,145.00","-292,145.00","-292,145.00",AN,64444,0.00,MtR,False,False,False,True
1,CONSULTATIO INVESTMENTS S.A.,148687,359,1359,"-214,500,000.00","-214,500,000.00","5,104,560.00","-153,430,000.00","-367,930,000.00","-148,325,440.00",Terceros,37126,0.00,MtR,False,False,False,True
2,PROSECURITIES S.A,163758,448,1448,"11,144.00","11,144.00","11,144.00","-1,916,600.00","-1,905,456.00","-1,905,456.00",Terceros,15916,0.00,MtR,False,False,False,True
3,NEXO AGENTE DE LIQUIDACION Y COMPENSACION INTE...,41772,393,1532,"3,314,538.99","3,314,538.99","3,314,538.99","-3,575,651.09","-261,112.10","-261,112.10",AN,64444,0.00,MtR,False,False,False,True
4,LARES CAPITAL S.A.,75392,389,59678,"-30,359,549.46","-30,359,549.46","-30,359,549.46","14,472,154.89","-15,887,394.56","-15,887,394.56",Propia,59678,"261,978,361.51",MtR,True,True,True,False


In [98]:
LOPS_ORDENADOS = LOPS.sort_values(["MiembroCompensadorCodigo","CuentaCompensacionCodigo","CuentaNeteoCodigo"])

### CALCULAR Y AGREGAR LOS TOTALES POR MC

In [99]:
lista_mc = list(LOPS_ORDENADOS["MiembroCompensadorCodigo"].drop_duplicates())
lista_cim = list(LOPS_ORDENADOS["CuentaCompensacionCodigo"].drop_duplicates())


In [100]:
mco=[]
lop_mc=[]
lopno_mc=[]

for mc in lista_mc:
    provisorio_mc = (LOPS_ORDENADOS[LOPS_ORDENADOS["MiembroCompensadorCodigo"]==mc])
    lop_verificado_mc = provisorio_mc["LOP_Verificado"].sum()
    lop_noverificado_mc = provisorio_mc["LOP_NoVerificado"].sum()
    lop_mc.append(lop_verificado_mc)
    lopno_mc.append(lop_noverificado_mc)
    mco.append(mc)


In [101]:

listas = [mco,lop_mc,lopno_mc]
dataframe_lops_mc = (pd.DataFrame(listas)).T.rename(columns={0:"MiembroCompensadorCodigo",1:"LOP Verificado MC",2:"LOP No Verificado MC"})
#dataframe_lops_mc = dataframe_lops_mc.rename(columns={"0":"MiembroCompensadorCodigo","1":"LOP Verificado MC","2":"LOP No Verificado MC"},inplace=True)

print(dataframe_lops_mc)

  MiembroCompensadorCodigo LOP Verificado MC LOP No Verificado MC
0                      359   -367,930,000.00      -148,325,440.00
1                      389    -15,887,394.56       -15,887,394.56
2                      393       -553,257.10          -553,257.10
3                      448     -1,905,456.00        -1,905,456.00


In [102]:
cim_lop=[]
lop_cim=[]
lopno_cim=[]

for cim in lista_cim:
    provisorio_cim = (LOPS_ORDENADOS[LOPS_ORDENADOS["CuentaCompensacionCodigo"]==cim])
    lop_verificado_cim = provisorio_cim["LOP_Verificado"].sum()
    lop_noverificado_cim = provisorio_cim["LOP_NoVerificado"].sum()
    lop_cim.append(lop_verificado_cim)
    lopno_cim.append(lop_noverificado_cim)
    cim_lop.append(cim)
    
listas = [cim_lop,lop_cim,lopno_cim]
dataframe_lops_cim = (pd.DataFrame(listas)).T.rename(columns={0:"CuentaCompensacionCodigo",1:"LOP Verificado CIM",2:"LOP No Verificado CIM"})
#dataframe_lops_mc = dataframe_lops_mc.rename(columns={"0":"MiembroCompensadorCodigo","1":"LOP Verificado MC","2":"LOP No Verificado MC"},inplace=True)

print(dataframe_lops_cim)

  CuentaCompensacionCodigo LOP Verificado CIM LOP No Verificado CIM
0                     1359    -367,930,000.00       -148,325,440.00
1                    59678     -15,887,394.56        -15,887,394.56
2                     1532        -261,112.10           -261,112.10
3                    35417        -292,145.00           -292,145.00
4                     1448      -1,905,456.00         -1,905,456.00


### MERGEAR DFS


In [103]:
LOPS_ORDENADOS = LOPS_ORDENADOS.merge(dataframe_lops_cim, on = "CuentaCompensacionCodigo", how = "left")
LOPS_ORDENADOS = LOPS_ORDENADOS.merge(dataframe_lops_mc, on = "MiembroCompensadorCodigo", how = "left")
LOPS_ORDENADOS


,ALyC,CuentaNeteoCodigo,MiembroCompensadorCodigo,CuentaCompensacionCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,...,SaldoPropia,EntidadBursatilDescripcion,DaCreditoIntraCuentaCompensacion,RecibeCreditoIntraCuentaCompensacion,DaCreditoInterCuentaCompensacion,RecibeCreditoInterCuentaCompensacion,LOP Verificado CIM,LOP No Verificado CIM,LOP Verificado MC,LOP No Verificado MC
0,CONSULTATIO INVESTMENTS S.A.,148687,359,1359,"-214,500,000.00","-214,500,000.00","5,104,560.00","-153,430,000.00","-367,930,000.00","-148,325,440.00",...,0.00,MtR,False,False,False,True,"-367,930,000.00","-148,325,440.00","-367,930,000.00","-148,325,440.00"
1,LARES CAPITAL S.A.,75392,389,59678,"-30,359,549.46","-30,359,549.46","-30,359,549.46","14,472,154.89","-15,887,394.56","-15,887,394.56",...,"261,978,361.51",MtR,True,True,True,False,"-15,887,394.56","-15,887,394.56","-15,887,394.56","-15,887,394.56"
2,NEXO AGENTE DE LIQUIDACION Y COMPENSACION INTE...,41772,393,1532,"3,314,538.99","3,314,538.99","3,314,538.99","-3,575,651.09","-261,112.10","-261,112.10",...,0.00,MtR,False,False,False,True,"-261,112.10","-261,112.10","-553,257.10","-553,257.10"
3,NEXO AGENTE DE LIQUIDACION Y COMPENSACION INTE...,143173,393,35417,0.00,0.00,0.00,"-292,145.00","-292,145.00","-292,145.00",...,0.00,MtR,False,False,False,True,"-292,145.00","-292,145.00","-553,257.10","-553,257.10"
4,PROSECURITIES S.A,163758,448,1448,"11,144.00","11,144.00","11,144.00","-1,916,600.00","-1,905,456.00","-1,905,456.00",...,0.00,MtR,False,False,False,True,"-1,905,456.00","-1,905,456.00","-1,905,456.00","-1,905,456.00"


In [104]:
cat = pd.read_excel("categorias.xlsx")
#LOPS_ORDENADOS = LOPS_ORDENADOS.merge(cat, on = "MiembroCompensadorCodigo", how = "left")
categorizado = LOPS_ORDENADOS
#categorizado.merge(cat, on = "MiembroCompensadorCodigo", how = "left
categorizado["MiembroCompensadorCodigo"] = categorizado["MiembroCompensadorCodigo"].astype(int)
cat["MiembroCompensadorCodigo"] = cat["MiembroCompensadorCodigo"].astype(int)
LOPS_ORDENADOS = categorizado.merge(cat, on = "MiembroCompensadorCodigo", how = "left")

In [105]:
def format_float(value):
    return f'{value:,.2f}'

pd.options.display.float_format = format_float
LOPS_ORDENADOS

,ALyC,CuentaNeteoCodigo,MiembroCompensadorCodigo,CuentaCompensacionCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,...,DaCreditoInterCuentaCompensacion,RecibeCreditoInterCuentaCompensacion,LOP Verificado CIM,LOP No Verificado CIM,LOP Verificado MC,LOP No Verificado MC,Categoría,1ERLIMITE,2DOLIMITE,3ERLIMITE
0,CONSULTATIO INVESTMENTS S.A.,148687,359,1359,"-214,500,000.00","-214,500,000.00","5,104,560.00","-153,430,000.00","-367,930,000.00","-148,325,440.00",...,False,True,"-367,930,000.00","-148,325,440.00","-367,930,000.00","-148,325,440.00",B,4700000,18800000,31300000
1,LARES CAPITAL S.A.,75392,389,59678,"-30,359,549.46","-30,359,549.46","-30,359,549.46","14,472,154.89","-15,887,394.56","-15,887,394.56",...,True,False,"-15,887,394.56","-15,887,394.56","-15,887,394.56","-15,887,394.56",B,4700000,18800000,31300000
2,NEXO AGENTE DE LIQUIDACION Y COMPENSACION INTE...,41772,393,1532,"3,314,538.99","3,314,538.99","3,314,538.99","-3,575,651.09","-261,112.10","-261,112.10",...,False,True,"-261,112.10","-261,112.10","-553,257.10","-553,257.10",D,1500000,6100000,10100000
3,NEXO AGENTE DE LIQUIDACION Y COMPENSACION INTE...,143173,393,35417,0.00,0.00,0.00,"-292,145.00","-292,145.00","-292,145.00",...,False,True,"-292,145.00","-292,145.00","-553,257.10","-553,257.10",D,1500000,6100000,10100000
4,PROSECURITIES S.A,163758,448,1448,"11,144.00","11,144.00","11,144.00","-1,916,600.00","-1,905,456.00","-1,905,456.00",...,False,True,"-1,905,456.00","-1,905,456.00","-1,905,456.00","-1,905,456.00",A,6700000,27000000,45000000


In [111]:
ESTADO = []
for i in range(len(LOPS_ORDENADOS)):
    if (LOPS_ORDENADOS.loc[i].LOP_Verificado) < (LOPS_ORDENADOS.loc[i].3ERLIMITE):
        print("Truuu")
    else:
        print("gudfd")


SyntaxError: invalid decimal literal (652048994.py, line 3)

In [107]:
LOPS_TEST = LOPS_ORDENADOS.groupby(["ALyC","CuentaCompensacionCodigo","CuentaNeteoCodigo"]).first()
LOPS_TEST
LOPS_TEST.to_excel("LOPS_AGRUPADOS.xlsx")


In [108]:
LOPS_ORDENADOS.to_excel("LOPS_CASIFINAL.xlsx",index=False)

## Situación LOP
#### Entonces deberiamos

1) Determinar total saldo verificado y no verificado por cuenta de neteo, sólo para
finalidad Márgenes 

2) Determinar margen del día por cuenta de neteo

3) Establecer el LOP verificado y no verificado de cada cuenta de neteo

Cuenta CEL

1) Determina la situación de LOP sumando los LOP verificado (y no verificado por otro lado) de cada cuenta de neteo interna y comparando con los límites de dicha CIM

Cuenta Tercero/CER/AN
1) Determina el LOP verificado y no verificado “total”, sumando los valorescorrespondientes de cada cuenta de neteo interna.
2) Si el total de LOP verificado es menor a 0, y cae dentro de algunos de los límites de LOP, buscamos el remanente de las cuentas propias del ALyC (es decir, los LOP verificados positivos) y en caso de poder, usamos de dicho valor la cantidad necesaria para “salir de LOP”. En este caso, el LOP verificado del AlyC “variaría” (sería un número menor) ya que estaríamos subvencionando a la CIM.
3) Si con el LOP del ALyC no “sale de LOP” no se usa nada y la CIM queda en la situación LOP actual. ¿Cómo usar el remanente de ALyC? Ordena las cuentas CIM por total LOP ascendente (del más negativo al más positivo), y dentro de cada CIM, ordenamos las cuentas de neteo por LOP ascendente. Finalmente, empezamos a utilizar el LOP del ALyC por CIM/Neteo “más negativa”.

Cuenta Propia
1) Determina el LOP verificado y no verificado “total”, sumando los valores correspondientes de cada cuenta de neteo interna.
2) Al LOP verificado y no verificado le “resto” lo que use para compensar otras CIM del ALyC y comparo contra los límites y determino la situación de LOP.
3) El “saldo ALyC” se define como el saldo de la CIM (compenso dentro de la cim propias con los LOP de las cuentas de neteo).

